In [80]:
# Part 1
Y = 0
X = 1


def parse(input_str):
    return [list(line) for line in input_str.splitlines()]


def n(yx):
    if yx[Y] == 0:
        return None

    y = yx[Y] - 1
    x = yx[X]
    if grid[y][x] == "#":
        return None

    return (y, x)


def e(yx):
    if yx[X] == WIDTH - 1:
        return None

    y = yx[Y]
    x = yx[X] + 1
    if grid[y][x] == "#":
        return None

    return (y, x)


def s(yx):
    if yx[Y] == HEIGHT - 1:
        return None

    y = yx[Y] + 1
    x = yx[X]
    if grid[y][x] == "#":
        return None

    return (y, x)


def w(yx):
    if yx[X] == 0:
        return None

    y = yx[Y]
    x = yx[X] - 1
    if grid[y][x] == "#":
        return None

    return (y, x)


def neighbours(cell):
    candidates = [n(cell), e(cell), s(cell), w(cell)]
    return [c for c in candidates if c]


def find_start():
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if grid[y][x] == "S":
                return (y, x)

    raise Exception("No start found")


def dump_grid(grid, visited=set()):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if (y, x) in visited:
                print("O", end="")
            else:
                print(grid[y][x], end="")
        print("")


# Go
grid = parse(input_str)
HEIGHT = len(grid)
WIDTH = len(grid[0])

steps = 64

frontier = set([find_start()])

for i in range(steps):
    next_frontier = set()
    for cell in frontier:
        for next in neighbours(cell):
            next_frontier.add(next)

    frontier = next_frontier
    # print(f"After {i + 1} steps, visited {len(frontier)} cells")

print("Part 1:", len(frontier))

Part 1: 42


In [144]:
# Part 2
import timeit

def get(y, x):
    return grid[y % HEIGHT][x % WIDTH]

def neighbours_infinite(yx):
    n = (yx[Y] - 1, yx[X])
    e = (yx[Y], yx[X] + 1)
    s = (yx[Y] + 1, yx[X])
    w = (yx[Y], yx[X] - 1)

    return [c for c in [n, e, s, w] if get(c[Y], c[X]) != "#"]

def neighbours_infinite_mod(yx):
    n = ((yx[Y] - 1) % HEIGHT, yx[X])
    e = (yx[Y], (yx[X] + 1) % WIDTH)
    s = ((yx[Y] + 1) % HEIGHT, yx[X])
    w = (yx[Y], (yx[X] - 1) % WIDTH)

    return [c for c in [n, e, s, w] if get(c[Y], c[X]) != "#"]
    
def n2(grid_yx, yx):
    y = (yx[Y] - 1) % HEIGHT
    x = yx[X]
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if y == HEIGHT - 1:
        grid_yx = inc_yx(grid_yx, Y, -1)

    return (grid_yx, (y, x))

def s2(grid_yx, yx):
    y = (yx[Y] + 1) % HEIGHT
    x = yx[X]
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if y == 0:
        grid_yx = inc_yx(grid_yx, Y, 1)

    return (grid_yx, (y, x))

def w2(grid_yx, yx):
    y = yx[Y]
    x = (yx[X] - 1) % WIDTH
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if x == WIDTH - 1:
        grid_yx = inc_yx(grid_yx, X, -1)

    return (grid_yx, (y, x))

def e2(grid_yx, yx):
    y = yx[Y]
    x = (yx[X] + 1) % WIDTH
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if x == 0:
        grid_yx = inc_yx(grid_yx, X, 1)

    return (grid_yx, (y, x))

def inc_yx(yx, field, delta):
    if field == Y:
       return (yx[Y] + delta, yx[X])
    else:
       return (yx[Y], yx[X] + delta)
    
def add_yx(yx1, yx2):
    if (yx2 == (0, 0)):
        return yx1
    else:
        return (yx1[Y] + yx2[Y], yx1[X] + yx2[X])
    

def neighbours_infinite_3(grid_yx, yx):
    candidates = [n2(grid_yx, yx), e2(grid_yx, yx), s2(grid_yx, yx), w2(grid_yx, yx)]

    return [c for c in candidates if c]

# Precompute neighbours for each cell
cache = {}
for y in range(HEIGHT):
    for x in range(WIDTH):
        cell = (y, x)
        cache[cell] = neighbours_infinite_3((0, 0), cell)
        # print(cell, "->", cache[cell])

# Go
steps = 500

# 500 -> 50s
# 1000 -> bailed after 5m

prev_frontier = set()
frontier = set([((0,0), find_start())])

tt = 0

for i in range(steps):
    next_frontier = set(prev_frontier)
    for (g, cell) in frontier:
        # cc = cache[cell]
        # print(cc)
        for (g_delta, cell_next) in cache[cell]:
            
            sx = timeit.default_timer()
            g_next = add_yx(g, g_delta)
            

            
            next_frontier.add((g_next, cell_next))
            tt += timeit.default_timer() - sx

    prev_frontier = frontier
    frontier = next_frontier
    # print(f"After {i + 1} steps, visited {len(frontier)} cells")

# print(frontier)

print("Part 2:", len(frontier))
print(tt)
# dump_grid(grid, frontier)

Part 2: 106776
13.980243133556542


In [72]:
# Test Input
input_str="""
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
""".strip()


In [69]:
# Input
input_str="""
...................................................................................................................................
.#...............#..#...........#....#................................................##......##..........#..............#.........
......#...................#...#..#..........#.#.......#.....................#.............##.....#....#......#..#........#.......#.
.#....#...#...#..#.................#.....#....#.#..#..........................#.....#...........#.............#....................
.#.............................##...##......#........#......................#.........##...#..#.................##.........#.#.....
....#.#....#.................#..#..#........#...#.#.#..#......................#........#....#..............#.....##................
....#.....#......#....#.##....#.##........#.....#......#........................#..........#.............#...................#.....
...........#...........##.#..........##.....#...#...##........#.................#.......#.............#......#..#..........#.......
..#...........#.............#...................................#.....................##...........#..............#.#....#.........
....#.....#.........#....#......##..............#...#........#.##....#........#..#....#..........#...................#..........#..
............#.........#............#.#...........#.........#.........................#......#..#......#..#..#.......#.....#........
.........#.................#........#..#.....#.............#........................#..#...#...#..#..#.....#...#..#......#....#....
..........#.#.#...............#.........#....##.#..........##......##...............#...#...###.##...#.......#..............#......
....#.........#...#........#.##.............#............#........#....##......................#..........#......#...#.......#.....
.....##..#......#...#........#................#............................#..........#............##..#...#.....#..#........##....
..#........#.#.###.....#....#..#.....#.....#..........................#.....#...........#.##..........#..........#...#...#.........
...................#.................................#.....#....#...#...#...................#..#......#....#.#.##................#.
.........#...#..#.#....#........#..........#................#.#....#...#.............................#..................#..#.#.....
...........#...#.#...................#..........................#..........#...................#...............#.###........#..#...
......#.......#...............#.........#.#...........#.####............#...#............#...##..##..........#............#........
..............................#..#.###..................#.....#......#..#.#..#...#..............#...#.....#.##..#.............#....
...............#.......#.#....#......#.#..........#..#.................#.#........#.......#.................#...............#...##.
............#.................#....................#............#....#.....#...............#........###........#.#..#.##...#.......
...#.....#.#..#.....#....#...#......#...........................#..#..#.......................#..............#..........#..........
.#....##..............#.##....#...#................#............#..........#......##....................##....#.#..#......#..##....
.....#.........#...##.#.#......................#....#...#...#.............#...................#...#..#...#...###......#...#....#...
........##......#.....#......................................#....###.#...#..........#.............#.....#.....#..........#.#......
........##.#...##.........##.....................#.#...........#........#.......................#...#.....#..............#.#.......
..........#.##.#..#..#..#.......#.....................................#.......##....#................#....#.....#...##.#....#..##..
..##.......#.........#..................##........#.....#.....................#.....#...................#.....#.#.........#........
....#........#.....#...................#.#..#..........................#...........#.............................#......#....#.....
..........................................#............#..#.......................#...#.#..#..................#.#............##....
...#..#....#......#....................................................................#..............#.#................#.......#.
..#.....##.............#.#.#...........#........#.................#..................#....................#..............#..###....
...##..#............#...#..#.........#..#.....#...........................#.....#....#...#...............#....#.............#...#..
...........#.......#...............................#..#.#...#.#...#...#........#.............##......................#...........#.
...............#..................#...............#.............#..#......................#......................##..........#.....
...........#..#....#.##...........#....#.##.#..........#...#.........#..........#.#..............................##.........#..#.#.
......#...#.#..##..................##................#####.#........#....#..#........#....#.....#................##..........#.....
....#.#........#.....#.......................##.......................#..#.#...........#.........................#..#..............
...#...........#..............#.......#.#....#......#...#....#.....#.....#...............#...........................#......#....#.
....#...........#..#.................#...##....#.............#..................#...........#.#.#............................#.....
........#..................#..#.#.........................###...#..#.#.#..##......#........##..#.....#..........#.....#.........#..
.....#.#.#....#...#..............#.#..#.......#.........###..#..........#.......#..#....................#....................##....
........#.......#...............##................#..............................#...#.#.....#....#.....##........#.#......#.......
.....#.##.....................#....#......###..#...........#........#....#..........#...#....#.........................#.......#...
..#.....#...#................#...##.#.......#....#.....#...............#.#....................#...#.......#..........#.............
........#.............#.#................#..#.....#....#.#......................#.....#.................##..#......................
..#..........#..........#..............#.........#..#.........#..............#.#.....#......###...#.....................#..........
......................#...#.#.#..#........#....###......##.........#.............#........#..#...#........#....................#...
..#..#...................#...#..#..#....#..##............#...#.............#.....#....##...............#...................##..#...
...#................###...............#...........#..............................#...#.........#.....#..#..#.................#...#.
....#.#..................##....#...#...#.........#....#.......#......#..........##..#..#..##.........#.....................#.......
......#..........#....#.....#.#.....#...#..........#.#......##.#......#............#.....#.....###.......#.#.#..#..................
...#.#.........##........#...#.....#......#...#.......#..................................#.........................................
..............#........................#.......###..#.......#.......#.#.......#....#.............#.......#...#.....#...............
.#..#..........#...........##..............#...#.........#.....##.#....................................#.......................#...
...................#........#....#.#...........#............#.#.#.##...##............#.#.....................#.....................
...........#....#........#...........#...#..##....#.#........#..#......#..##....................#...#..............................
.............#..........#.....#.......##......#......##.....#......##............................##.........#.........#.#..........
..............#.#...#.....#.....##..............#.......#.#..#.#......#...................#....#........#..........................
.....................##..#.#.....#..........#......#......#..#....#.#.............#...#......#..#..................................
...............#.#.##........#.........#..........#.....#............#...#.......##.........#........##....#..#....................
...........##....#.............#.................#...#..#...#........#...#.#..#..#.................#...#.....#.....................
..................................#........#..........#...........##.#.##....#.#..#............#.#.............#...................
.................................................................S.................................................................
......#..................###.##........#..#.....#....................#............#....#.#...#.#....#.###......#....##.......#.....
......................#.##......#.#......#..#....#.....................#...........#.....#..............#.#..#......#..............
..............................#..#............#......##.#.........#........#........#........................#.....................
..........#........#..........##.#.#..#...................#.#.#............#...............#...#.......#....#...##.................
............#.....#..........#.#.......#.#.........#....#.#.....#....#..........#.......#..............#........#..#....##.........
............#.............................#.................#......#..................#..#.....#..#...............#...##...........
.................###....#...#.#..............#.#..............#.......#...##....#.....#............#...#..##.##........#.......#.#.
....................#..#..##.#.#..##.....##...............#........#......##................#......................................
...................#.......#....#....#.......#..#.............#..........#.#..........#.##.............#...........................
....................................#..#...#....#......##..#.......#...#................#.....#....#.#.......#...#.#...............
.##...##.......#........#...#...#......#...#..............#..............##..##...#.........#...##...#.....................#..#....
.#....#.............#.#....#....#...............#..#....#..........#.......#......#.....#.................#.....................#..
.#............................#......##......#..#.#........#.....................#.#..#..##..#......#....................#..#......
.......#............#....#....#...#..#........#...............#......#.......#..................#......#...#.............#.........
..#.................#.####..#...........#...........#..............#..#.##..##....#................#.....#.................#...#...
.....#..............#...#........#................#....##.##..#..............#..##......#..#.........#................##...#...#...
.........##..........#.......#..........#.#....#......#...........###...........#.......................#..#.#................#.#..
.......#......#.......#......................................#..............#.................#..#........#..........#..........#..
.......#...#..............#.......##...........#.#.#......#.........#.......#....#........#..#..#........#.................#.......
.#.....##.#.#.................#.###...................##......#....##..##................#...............#.........................
..........##..#............#.#............#.......#..#.............#.......##.....##..#...##...#...................#...##.#...#....
.##...#.....##.............#.........#...............#.............................#...........#....................#.....##.......
..........##.#.............#............#...#..#....#...........#.....#..........#...#.......#..#.#................................
..#.....................................#.....................#......#...............#...#........#......................#.##......
.........#.#....#....#...............#......#...#........##...........#.......#....#....#...#......#.............#.....#....##.....
.............#....................#...........#...................#.#...................#.#..#....................#..........#.....
.....#.............#..#...................#................#......#...#.......#......#....#..##...................#.#.##.....#..#..
.#..##...#........#......................#...##.#..##.........................#............#.#............#..#.....................
.#...................#................#..............##...........#........#.#.....#.#...#.................#...##...#....#..#......
....#.......#..#...#.#...#.............#..#.##...#....#....#.......................#......#....#........#..#.......................
........###.......##..................#.......#....#.##....#..................##..........................#..#........#.........#..
.##.......##.....#.........#..........#......#...............#........#.#..#..........#...............#.#...#..........#...........
.#...##............#.........#..........##.....#..#......#......#....##...#...##.....#..#..............#.#.......##....#.#.#...#.#.
...........#...........................#..........#.#..........#....#..........#....................#.....##.............#.......#.
.........#..................##............#..................#................#.#.......#.............###..#....#.......#........#.
.......#...............#................#........#.....#.................##......#.......................#...............#.........
...##............#......#....................#...##.............#......#....#.....##.........................#.............#.......
.#.......#.#.#..#.......#......#..............#...#.........#......##.##.....#.....................#.....#.....#......##......#....
.#...................#.#....#.....#..........#..#...#.......................#...#....#.....................#....#...###.....#......
......#.....#......#....#.#.................#.........#.##...............##...................#..#......#..#.............#.........
..#........#....#.#............................................#......#...#.......#.....................#...................#......
.........#...#.#.....................................#....#..........................................##...#........................
..#...............##.#............#....#........##.#....#..................#...#...........#...#..#..#....#..........#....#....#...
............#.#.......###.......##.##.....................#...#.#...#..#....#...##............#.#.#..............#..#.#...#........
................................#..#.#....................#..#.#......#..###...............#..#...#...##.#.......#.......#.........
....#........#.......#..#..........##.........................#....#......##..#................#......#..#......#.#......#.#.......
.#..............#.#....#..........#.#....................#......#............................#....#......##...#.#.......#..........
....#.....##...#........#..#...........#...#.........#...............#.................#.#..#.................#..............#.....
...........................................##.............#.#.....#......#................#...#........#.......#..#.#..............
......#.....#..#.......##....#..........#......................#............#..........#...#...............#..#..#....#...#...#....
..........#...##.....................#.................##...............##.............#..##...........#.#............#.....##.....
.#.#........................#...#.....#...........................#........................#........#.....#.#.#...........#........
..#...#....................#..#...#.........#...#........#.#...#...................#..#.#.....................#...........#........
...#.#.......#...#.................#......#...#...#....................##.................#..........##.......#..#.....#........#..
..................#.......#................#...............#.#....#....#......................#.....#.......#.........##...#.......
..#.......#.#..............#..###........##.#.................................##..........#........#..........#.....#.....#........
.........##.....#.........#..........#...........#.#.................#.........#..............#...............#......#.#...##......
...............#.......#.#...#...........#.....#...#........................#..............#.......#.#......#....#......#....#.....
.................##.#.#........##..........................................#....#.........##......#.......#..#..................#..
...#..##......#.....#.............##.#..##....#...............................##......###.#....###.#.........#...........#...#.##..
..#..#..............#.....#.....#..#..##.......#..##.....#.................#...#................#..#........#...##...#.............
.#..................#............#.#..................................................#......#.#.......###...........#........##.#.
............#..........#....#.....#...#.....#........#.#.........................#.#....#....#.##..........#......#................
....#....#..#....................#.....#........##......#...#.........##.....#...##...#...##.##..............#....#......#..#...#..
...................................................................................................................................
""".strip()